In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pymetis
import networkx as nx
import time
from networkx.algorithms import community
from random import shuffle
import math
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch_geometric.data import Data
import torch.optim as optim
import pywt
from scipy.stats import norm
import scipy.interpolate as interp
import collections

In [2]:
def open_data(file_path):
    file = open(file_path,"rb")
    raw_data = pickle.load(file)
    
    return raw_data

In [3]:
def read_data(file_path):
    with h5py.File(file_path, 'r') as f:
    # 查看文件中所有的数据集名称
        dataset_names = list(f.keys())
        print("Datasets in the file:", dataset_names)
        data_dict = {}
        for name in dataset_names:
            if f[name].shape == (): 
                data = f[name][()] 
            else:
                data = f[name][:]
        
            data_dict[name] = data
            
        return data_dict

In [4]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [5]:
def save_data_ash5(data, file_path):
    with h5py.File(file_path, 'w') as f:
        keys =list(data.keys())
        for i in range(len(keys)):
            f.create_dataset(keys[i], data=data[keys[i]])

In [6]:
def renumber_subgraph(nodes, edge_index):
    unique_nodes = torch.unique(nodes)
    new_node_ids = torch.arange(len(unique_nodes))
    node_mapping = {old_id.item(): new_id.item() for old_id, new_id in zip(unique_nodes, new_node_ids)}
    
    new_edge_index = torch.tensor([
        [node_mapping[edge_index[0, i].item()], node_mapping[edge_index[1, i].item()]]
        for i in range(edge_index.size(1))
    ]).t()
    
    return new_edge_index

In [7]:
def precompute_dist_data(edge_pairs):
#距离是1/实际距离
    graph = nx.Graph()
    edge_pairs = edge_pairs.transpose(1,0).tolist()
    graph.add_edges_from(edge_pairs)
    H = nx.Graph()
    
    for node in sorted(graph.nodes):
        H.add_node(node)
        
    for edge in graph.edges:
        H.add_edge(*edge)  
        
    n = np.unique(edge_pairs).shape[0]
    dists_array = np.zeros((n, n))
        # dists_dict = nx.all_pairs_shortest_path_length(graph,cutoff=approximate if approximate>0 else None)
        # dists_dict = {c[0]: c[1] for c in dists_dict}
    dists_dict =dict(nx.all_pairs_shortest_path_length(H))
    
    for i, node_i in enumerate(H.nodes()):
        shortest_dist = dists_dict[node_i]
        for j, node_j in enumerate(H.nodes()):
            dist = shortest_dist.get(node_j, -1)
            if dist!=-1:
                dists_array[i, j] = 1 / (dist + 1)
    return torch.tensor(dists_array)

In [8]:
def get_random_anchorset(n,c=0.5): #n是一个图的所有点数量
    m = int(np.log2(n))
    copy = int(c*m)
    anchorset_id = []
    for i in range(m):
        anchor_size = int(n/np.exp2(i + 1))
        for j in range(copy):
            anchorset_id.append(np.random.choice(n, size=anchor_size, replace=False))
    return anchorset_id

In [9]:
def get_dist_max(edge_pairs, max_ach_num, max_subgraph_node_num):
    dist = precompute_dist_data(edge_pairs)
    n=np.unique(edge_pairs).shape[0]
    anchorset_id = get_random_anchorset(n,c=0.5)
    dist_max = torch.zeros((max_subgraph_node_num, max_ach_num))
    dist_argmax = torch.zeros((max_subgraph_node_num, max_ach_num)).long()
    for i in range(len(anchorset_id)):
        temp_id = torch.as_tensor(anchorset_id[i], dtype=torch.long)
        dist_temp = dist[:, temp_id]
        dist_max_temp, dist_temp_argmax = torch.max(dist_temp, dim=-1)#找到每一行距离最大的anchor对应的列索引
        dist_argmax_temp = temp_id[dist_temp_argmax]
        #-------------------------------------------------------
        dist_max[:dist.shape[0],i] = dist_max_temp
        dist_argmax[:dist.shape[0],i] = dist_argmax_temp
        
    return dist_max, dist_argmax

In [10]:
def Dist_Max_Dist_Argmax_Prepare(edge_index, max_ach_num, max_subgraph_node_num, layer_num):
    dists_max_sets = []
    dists_argmax_sets = []
    for i in range(layer_num):
        dists_max, dists_argmax = get_dist_max(edge_index, max_ach_num, max_subgraph_node_num)
        print(f"At Layer {i}, Dists_argmax shape and max node ID")
        print(dists_argmax.shape)
        print(torch.max(dists_argmax))
        print("-----------------------------")
        dists_max_sets.append(dists_max)
        dists_argmax_sets.append(dists_argmax)
    return torch.stack(dists_max_sets), torch.stack(dists_argmax_sets)

In [11]:
def subgraph_information(city_names, root_path):
    edge_pair_dictionary = {}
    max_ach_num = 0
    max_subgraph_node_num = 0 
    
    for name in city_names:
        edge_pairs = []
        dirs = os.listdir(root_path + name + "/edge_pair/")
        
        #-------------获取每个字图的edge pairs----------
        edge_pairs = [torch.tensor(open_data(root_path + name + "/edge_pair/" + each_file),dtype=torch.long) for each_file in dirs]
        edge_pair_dictionary[name + "_edge_pair"] = edge_pairs
        
        #------------获取每个子图的节点-----------------
        comm_node_list = [torch.unique(edge_pair, sorted = True) for edge_pair in edge_pairs]
        edge_pair_dictionary[name + "_nodes"] = comm_node_list
        
        #--------------计算子图的节点数-----------------
        graph_num = [sub_graph.shape[0] for sub_graph in comm_node_list]
        edge_pair_dictionary[name + "subgraph_node_num"] = graph_num
        
        #---------Calculate the anchor set num of each comm----------------
        ach_set_nums = [int(0.5*int(np.log2(node_num.shape[0])))* int(np.log2(node_num.shape[0]))
                        for node_num in comm_node_list]
        edge_pair_dictionary[name + "_anchor_set_num"] = ach_set_nums
        
        #---------获取每个城市的节点数-----------------
        city_node_num = sum([comm_node.shape[0] for comm_node in comm_node_list])
        edge_pair_dictionary[name + "_city_node_num"] = city_node_num
        
        if max_ach_num <= max(ach_set_nums):
            max_ach_num = max(ach_set_nums)
        if max_subgraph_node_num <= max(graph_num):
            max_subgraph_node_num = max(graph_num)
        
    return edge_pair_dictionary, max_ach_num, max_subgraph_node_num

In [12]:
city_names = ["Barcelona", "Antwerp","Bangkok"]
root_path = "D:/ThesisData/processed data/"

In [18]:
edge_pair_dictionary, max_ach_num, max_subgraph_node_num = subgraph_information(city_names, root_path)

In [19]:
for i in range(len(city_names)):
    city_nodes = edge_pair_dictionary[city_names[i] + "_nodes"]
    print(f"City {city_names[i]}")
    max_sub_nodes = [torch.max(edge_pair_dictionary[city_names[i] + "_nodes"][j]) for j in range(len(city_nodes))]
    node_nums = [edge_pair_dictionary[city_names[i] + "subgraph_node_num"][j]for j in range(len(city_nodes))]
    print(node_nums)

City Barcelona
[1252, 1070, 1042, 1004, 1017, 1162, 1093, 1130, 1035, 1067, 1501, 1185, 1056, 1301]
City Antwerp
[1450, 1366, 1446, 1264, 1439, 1478, 1325, 1459, 1213, 1331, 1265, 1511]
City Bangkok
[1699, 1529, 1461, 1527, 1582, 1548, 1577, 1419, 1617, 1333, 1465, 1498, 1674, 1535]
City Moscow
[1392, 1349, 1409, 1406, 1329, 1300, 1257, 1528, 1278, 1311, 1334, 1375, 1337, 1272, 1579, 1386]


In [20]:
print(max_ach_num)
print(max_subgraph_node_num)

50
1699


In [21]:
all_nodes = edge_pair_dictionary["Barcelona_nodes"]+edge_pair_dictionary["Antwerp_nodes"]+edge_pair_dictionary["Bangkok_nodes"]
min_id = [torch.min(comm_nodes).item() for comm_nodes in all_nodes]
print(min_id)

[14096, 63, 43, 2897, 3749, 11466, 7293, 9793, 8466, 6420, 1, 8244, 4619, 30, 10620, 13, 1, 9399, 12779, 74, 2965, 5412, 6657, 35, 32, 8201, 1, 64, 3708, 87, 2319, 2900, 4646, 12407, 15339, 8956, 14968, 9281, 10457, 13526]


In [33]:
city_name = "Moscow"
data_dict = read_data(root_path + city_name + "/input_target/input_target.h5")

Datasets in the file: ['city_node_num', 'max_vals', 'min_vals', 'period_template', 'target_template', 'trend_template']


In [31]:
#-----------Calculate distance_matrix----------
def dist_max_argmax_cal(subgraph_nodes, subgraph_edge_pairs, max_ach_num, max_subgraph_node_num, layer_num):
    subgraph_dist_max = []
    subgraph_dist_argmax = []
    for i in range(len(subgraph_nodes)):
        new_edge_index = renumber_subgraph(subgraph_nodes[i], subgraph_edge_pairs[i])
        print(f"for subgraph {i}, our max edge index is {torch.max(new_edge_index)}")
        dist_max_tensors, dist_argmax_tensors = Dist_Max_Dist_Argmax_Prepare(new_edge_index, max_ach_num, max_subgraph_node_num, layer_num)
        subgraph_dist_max.append(dist_max_tensors)
        subgraph_dist_argmax.append(dist_argmax_tensors)
    
    #torch.tensor(subgraph_dist_max) Shape: SubgraphNum, LayerNum, Max_Subgraph_Node_num, Max_Ach_Num
    return torch.stack(subgraph_dist_max).to(dtype=torch.float), torch.stack(subgraph_dist_argmax).to(dtype=torch.int)

In [34]:
layer_num = 3
subgraph_edge_pairs = edge_pair_dictionary[city_name + "_edge_pair"]
subgraph_nodes = edge_pair_dictionary[city_name + "_nodes"]

subgraph_dist_max, subgraph_dist_argmax =dist_max_argmax_cal(subgraph_nodes, subgraph_edge_pairs,
                                                             max_ach_num, max_subgraph_node_num, layer_num)
dist_dictionary = {"dist_max": subgraph_dist_max, "dist_argmax": subgraph_dist_argmax}

save_path = root_path + city_name + "/input_target/dist_dictionary.h5"
save_data(dist_dictionary, save_path)


for subgraph 0, our max edge index is 1391
At Layer 0, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1391)
-----------------------------
At Layer 1, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1391)
-----------------------------
At Layer 2, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1391)
-----------------------------
for subgraph 1, our max edge index is 1348
At Layer 0, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1348)
-----------------------------
At Layer 1, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1348)
-----------------------------
At Layer 2, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1348)
-----------------------------
for subgraph 2, our max edge index is 1408
At Layer 0, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(1408)
-----------------------------
At Layer 1, Dists_argmax shape and max node ID
torch.Size([1699, 50])
tensor(140